In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import bb_rt_helper

In [3]:
import importlib
importlib.reload(bb_rt_helper)

<module 'bb_rt_helper' from '/home/jovyan/data-analyses/one_hundred_recs/bb_rt_helper.py'>

# 100 Recs - D6

## 1 - Corridor Improvements

In [4]:
bb_rt_helper.fm_from_bbutils?

Signature:
bb_rt_helper.fm_from_bbutils(
    ct_dist,
    category,
    get_sorted=False,
    fm_dict={},
    analysis_date=datetime.date(2022, 5, 4),
)
Docstring:
ct_dist: str, Caltrans district, formatted as in "11 - San Diego"
category: str, reccomendation category, "corridor" or "hotspot"
get_sorted: use better_bus_utils.get_sorted_transit_routes instead of 
corridor/hotspot dedicated function. Default False, override if few/no
suitable routes returned.
fm_dict: dictionary of rt_analysis.rt_filter_map_plot.RtFilterMapper instances,
if some are already loaded (for example when running for hotspot after corridor)
File:      ~/data-analyses/one_hundred_recs/bb_rt_helper.py
Type:      function


In [5]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist='06 - Fresno', category='corridor')

06 - Fresno corridor empty with default util, retrying with get_sorted


loading filter/mapper for 126
found parquet
loading filter/mapper for 188
found parquet
no route filter on 188
loading filter/mapper for 146
no filtermapper for 146
loading filter/mapper for 346
no filtermapper for 346
loading filter/mapper for 474
no filtermapper for 474
{126: <rt_analysis.rt_filter_map_plot.RtFilterMapper object at 0x7f32443c4a30>, 188: <rt_analysis.rt_filter_map_plot.RtFilterMapper object at 0x7f32443c4a60>}


In [6]:
corr_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    corr_speed_views += [fm_dict[key].detailed_map_view.copy()]

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic 

### Operator: Madera County Connection

In [7]:
# manually add route filter via trip ids
mcc_trips = (fm_dict[188].rt_trips
             >> filter(_.route_id=="3615")
            ).trip_id
mcc_trips

16    t_1132965_b_26467_tn_0
17    t_1132966_b_26467_tn_0
18    t_1132970_b_26467_tn_0
19    t_1132973_b_26467_tn_0
20    t_1132968_b_26467_tn_0
21    t_1132971_b_26467_tn_0
Name: trip_id, dtype: object

In [8]:
fm_dict[188].rt_trips.route_short_name.isnull().all()

True

In [9]:
fm_dict[188].set_filter(trip_ids = mcc_trips)

In [12]:
m_corr = fm_dict[188].segment_speed_map(no_title = True)

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [13]:
m_corr

### Operator: GET Bus

In [14]:
m_corr = fm_dict[126].segment_speed_map()

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [15]:
m_corr

## Draft Corridor Recs, SDMTS, D11

* Entire Route 28 (nn mi, nn speed-based delay, nn schedule-based delay)
* Route 10, Goldfinch to 54 ()
    * paralleled by faster Rapid 215 E of Park
* Route 27, Mission to Moraga
* Route 12, Broadway to Euclid

## Route 28

In [ ]:
shape_id = '28_2_48'
stop_range = [2, 18]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].set_filter(route_names=['28'])

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_28 = fm_dict[278].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors = [corr_28]

## Route 10

In [ ]:
shape_id = '10_2_68'
stop_range = [6, 21]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].set_filter(route_names=['10'])

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_10 = fm_dict[278].corridor.copy()
corridors += [corr_10]

## Route 27

In [ ]:
shape_id = '27_2_81'
stop_range = [3, 16]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].set_filter(route_names=['27'])

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_27 = fm_dict[278].corridor.copy()
corridors += [corr_27]

## Route 12

In [ ]:
shape_id = '12_2_13'
stop_range = [3, 31]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].set_filter(route_names=['12'])

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_12 = fm_dict[278].corridor.copy()
corridors += [corr_12]

## 2 - Hotspot Improvements

In [ ]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist='11 - San Diego', category='hotspot', fm_dict = fm_dict)

In [ ]:
hs_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    hs_speed_views += [fm_dict[key].detailed_map_view.copy()]

In [ ]:
m_hs = fm_dict[278].segment_speed_map()

In [ ]:
m_hs

# Draft Hotspot Recs, SDMTS, D11

## Route 27 (Genesee)

In [ ]:
shape_id = '27_2_81'
stop_range = [19, 23]

In [ ]:
fm_dict[278].set_filter(route_names=['27'])

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_27 = fm_dict[278].corridor.copy()
hotspots = [hs_27]

## Route 10 (I-5)

In [ ]:
shape_id = '10_3_67'
stop_range = [21, 22]

In [ ]:
fm_dict[278].set_filter(route_names=['10'])

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_10 = fm_dict[278].corridor.copy()
hotspots += [hs_10]

## Route 14 (I-8)

In [ ]:
shape_id = '14_2_96'
stop_range = [28, 29]

In [ ]:
fm_dict[278].set_filter(route_names=['14'])

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_14 = fm_dict[278].corridor.copy()
hotspots += [hs_14]

## Route 3 (Commercial to 24th)

In [ ]:
shape_id = '3_0_90'
stop_range = [27, 32]

In [ ]:
fm_dict[278].set_filter(route_names=['3'])

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_3 = fm_dict[278].corridor.copy()
hotspots += [hs_3]

## Map Testing

In [ ]:
all_speed_segs = pd.concat(hs_speed_views)
all_corr_segs = pd.concat(corr_speed_views)
# full join to preserve all unique segments, since they can differ
# from hotspot to corridor (or indeed across operators)
all_segs = all_speed_segs >> full_join(_, all_corr_segs)

In [ ]:
bb_rt_helper.bb_map_all(hotspots, corridors, all_segs, district = '11 - San Diego')